[View in Colaboratory](https://colab.research.google.com/github/pmanlukas/colab/blob/master/prototype_model.ipynb)

# Prototype: deploy model

This notebook is used to test if the serialized model can be used for usage in a web service.

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18396 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

Setting up libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8.1) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [4]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [6]:
!ls /content/drive/openapi-data/

CNN_cat_eval.h5        glove.6B.zip		PoC Classifier Path.odt
CNN_cat.h5	       LSTMCNN_cat.h5		structure_labels0518.csv
CNN_cat_weights.h5     LSTMCNN_cat_weights.h5	structure_labels.csv
CNN_path.h5	       mnist.py			tokenizer_cat
CNN_path_weights.h5    new_path_labels0518.csv	tokenizer_path
CNN_struct.h5	       path_labels0518.csv	tokenizer_struct
CNN_struct_weights.h5  path_labels.csv		weights.best.hdf5
drive		       pickle			wordvector


In [0]:
import os
os.chdir("drive/openapi-data")

In [0]:
from collections import Counter
from datetime import datetime

import pickle
import json
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [0]:
cat_label = dict()

with open('pickle/categories_dict0518.pkl', 'rb') as handle:
    cat_label = pickle.load(handle)

In [0]:
cat_data = dict()

with open('pickle/spec_category0518.pkl', 'rb') as handle:
    cat_data = pickle.load(handle)

In [0]:
keys = list(cat_data.keys())

In [12]:
cat_data[keys[0]]

{'basePath': '/forex-quotes',
 'host': '1forge.com',
 'info': {'contact': {'email': 'contact@1forge.com',
   'name': '1Forge',
   'url': 'http://1forge.com'},
  'description': 'Stock and Forex Data and Realtime Quotes',
  'title': '1Forge Finance APIs',
  'version': '0.0.1',
  'x-apisguru-categories': ['financial'],
  'x-logo': {'backgroundColor': '#24292e',
   'url': 'https://api.apis.guru/v2/cache/logo/http_1forge.com_logo.png'},
  'x-origin': [{'format': 'swagger',
    'url': 'http://1forge.com/openapi.json',
    'version': '2.0'}],
  'x-preferred': True,
  'x-providerName': '1forge.com'},
 'paths': {'/quotes': {'get': {'description': 'Get quotes',
    'externalDocs': {'description': 'Find out more',
     'url': 'http://1forge.com/forex-data-api'},
    'responses': {'200': {'description': 'A list of quotes'}},
    'summary': 'Get quotes for all symbols',
    'tags': ['forex', 'finance', 'quotes']}},
  '/symbols': {'get': {'description': 'Symbol List',
    'externalDocs': {'descripti

In [13]:
cat_texts = []   # list of text samples
for cat in cat_data:
  stringg = json.dumps(cat_data[cat])
  cat_texts.append(stringg)

print(str(len(cat_texts)))

944


In [14]:
cat_info = dict()
for cat in cat_data:
    info = cat_data[cat]['info']
    cat_info[cat] = info
print(cat_info.keys())

dict_keys(['1forge.com', '6-dot-authentiqio.appspot.com', 'adafruit.com', 'adobe.com:aem', 'adyen.com:CheckoutService', 'adyen.com:PaymentService', 'adyen.com:RecurringService', 'afterbanks.com', 'agco-ats.com', 'airport-web.appspot.com', 'amazonaws.com:AWSMigrationHub', 'amazonaws.com:acm', 'amazonaws.com:acm-pca', 'amazonaws.com:alexaforbusiness', 'amazonaws.com:apigateway', 'amazonaws.com:application-autoscaling', 'amazonaws.com:appstream', 'amazonaws.com:appsync', 'amazonaws.com:athena', 'amazonaws.com:autoscaling', 'amazonaws.com:autoscaling-plans', 'amazonaws.com:batch', 'amazonaws.com:budgets', 'amazonaws.com:ce', 'amazonaws.com:cloud9', 'amazonaws.com:clouddirectory', 'amazonaws.com:cloudformation', 'amazonaws.com:cloudfront', 'amazonaws.com:cloudhsm', 'amazonaws.com:cloudhsmv2', 'amazonaws.com:cloudsearch', 'amazonaws.com:cloudsearchdomain', 'amazonaws.com:cloudtrail', 'amazonaws.com:codebuild', 'amazonaws.com:codecommit', 'amazonaws.com:codedeploy', 'amazonaws.com:codepipelin

In [15]:
texts_info_cat = []   # list of text samples
for cat in cat_info:
  strJson = json.dumps(cat_info[cat])
  texts_info_cat.append(strJson)

print(str(len(texts_info_cat)))

944


In [16]:
labelsCat = list()

for labc in cat_label:
  labelsCat.append(cat_label[labc])
  
print(str(len(labelsCat)))

944


In [0]:
from sklearn import model_selection, preprocessing, linear_model

#encoder for labels of category
encoder = preprocessing.LabelEncoder()
labelsCat = encoder.fit_transform(labelsCat)

In [19]:
labelsC = labelsCat  # list of label ids
print(type(labelsC))
print(labelsC)

<class 'numpy.ndarray'>
[11 23 13 16 21 21 21 11  6 31  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  5 18 29  6  6  5 20 28 28 28 28 28 28  5 28 28 28 28 28
 28 28 28 28 28 28 28 28 28  5 22 22 17 11 20 23  3 31  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  2  2

In [0]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [21]:
encoder = LabelBinarizer()
encoder.fit(labelsC)
yc_train = encoder.transform(labelsC)

yc_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [22]:
import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using TensorFlow backend.


In [0]:
#Values are adjusted to fit to input and labels
MAX_SEQUENCE_LENGTH = 75000
MAX_NUM_WORDS = 40000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [24]:
tokenizer = Tokenizer(nb_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts_info_cat)
sequences = tokenizer.texts_to_sequences(texts_info_cat)
pickle.dump(tokenizer, open('tokenizer_cat', 'wb'))

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = yc_train #keras.utils.to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

/usr/local/lib/python3.6/dist-packages/keras/preprocessing/text.py:172: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 6891 unique tokens.
Shape of data tensor: (944, 75000)
Shape of label tensor: (944, 32)


In [0]:
from keras.models import load_model
new_model = load_model('CNN_cat.h5')


In [33]:
print(x_val[0])
print(x_val[0].shape)
type(x_val[0])

[   0    0    0 ...   30 1177    3]
(75000,)


numpy.ndarray

In [0]:
y_softmax = new_model.predict(x_val)

In [0]:


y_test_1d = []
y_pred_1d = []

for i in range(len(y_val)):
    probs = y_val[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [26]:
print(y_test_1d)
print(y_pred_1d)

[13, 11, 2, 9, 2, 17, 31, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 25, 22, 17, 2, 2, 5, 2, 2, 2, 11, 2, 2, 13, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 17, 11, 20, 14, 6, 2, 20, 11, 2, 2, 20, 2, 2, 2, 2, 2, 2, 20, 28, 20, 30, 6, 2, 2, 24, 2, 20, 2, 2, 2, 2, 17, 28, 28, 27, 2, 2, 2, 2, 20, 5, 2, 2, 2, 2, 2, 2, 2, 30, 2, 5, 20, 2, 10, 2, 22, 17, 17, 17, 6, 2, 14, 2, 2, 2, 2, 2, 17, 31, 11, 2, 2, 20, 28, 2, 11, 2, 25, 22, 20, 6, 2, 16, 29, 27, 2, 17, 2, 2, 20, 2, 2, 17, 2, 20, 10, 2, 2, 2, 2, 10, 2, 2, 2, 6, 2, 2, 2, 2, 17, 2, 2, 17, 17, 2, 2, 0, 2, 6, 2, 30, 2, 9, 17, 11, 10, 20, 13, 2, 6, 6, 15, 2, 18, 6, 24, 2, 2, 2, 2, 2, 5, 2, 2]
[13, 11, 2, 8, 2, 17, 31, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 2, 2, 25, 25, 17, 2, 2, 5, 2, 2, 2, 11, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 10, 17, 11, 20, 14, 6, 2, 20, 11, 2, 2, 20, 2, 2, 2, 2, 2, 2, 20, 28, 20, 1, 6, 2, 2, 24, 2, 20, 2, 2, 2, 2, 17, 28, 28, 27, 2, 2, 2, 2, 20, 5, 2, 2, 2, 2, 2, 2, 2, 6, 2, 5, 20, 2, 10, 2, 22, 17, 17, 17, 6, 2, 14, 2, 2, 2, 2, 2, 17

In [29]:
y_softmax 

array([[6.1472051e-04, 4.5468949e-02, 4.9776742e-03, ..., 3.3660362e-07,
        4.9538713e-02, 1.0870144e-03],
       [4.5755162e-04, 1.9044588e-05, 6.0807552e-06, ..., 1.4756907e-07,
        4.9300366e-03, 1.9909007e-02],
       [3.8384407e-08, 5.0640436e-05, 9.9797100e-01, ..., 7.6277691e-12,
        7.8754465e-07, 1.2169907e-07],
       ...,
       [5.6377787e-04, 1.2748393e-05, 1.6517039e-07, ..., 9.2121880e-08,
        9.4887121e-03, 2.0340339e-03],
       [2.5583531e-06, 8.4661209e-04, 9.8119467e-01, ..., 4.6629939e-10,
        1.5611249e-05, 6.4090013e-06],
       [4.7834178e-08, 5.9329763e-05, 9.9761152e-01, ..., 8.1514066e-12,
        8.0438383e-07, 1.0948183e-07]], dtype=float32)